In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        redeemUnderlyings(where: {timestamp_lte:""" + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            pool {
                id
            }
            user {
                id
            }
            to {
                id
            }
            reserve {
                symbol
            }
            amount
            timestamp
        }
    }
    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})
    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["redeemUnderlyings"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue
    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["user"] = transaction["user"]["id"]
        transaction["to"] = transaction["to"]["id"]
        transaction["reserve"] = transaction["reserve"]["symbol"]
        transaction["pool"] = transaction["pool"]["id"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1

with open('redeemUnderlying.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(transactions)
        

1000 transactions found at timestamp 1654016400
998 transactions found at timestamp 1653678545
998 transactions found at timestamp 1653385846
999 transactions found at timestamp 1653040788
999 transactions found at timestamp 1652783949
999 transactions found at timestamp 1652575288
999 transactions found at timestamp 1652415366
998 transactions found at timestamp 1652341238
999 transactions found at timestamp 1652290642
999 transactions found at timestamp 1652143768
998 transactions found at timestamp 1652036682
999 transactions found at timestamp 1651824717
999 transactions found at timestamp 1651425457
999 transactions found at timestamp 1651020447
999 transactions found at timestamp 1650642871
999 transactions found at timestamp 1650232930
999 transactions found at timestamp 1649751790
999 transactions found at timestamp 1649384646
999 transactions found at timestamp 1648938663
999 transactions found at timestamp 1648495712
999 transactions found at timestamp 1648090005
999 transact

In [2]:
!wc -l redeemUnderlying.csv
# 208755

211730 redeemUnderlying.csv


In [3]:
from google.colab import files
files.download('redeemUnderlying.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>